# Extract Data

In [1]:
root_path = '/home/santosh/Work/Datasets/CORD-19-research-challenge/'
paths = [
    'biorxiv_medrxiv/biorxiv_medrxiv/',
        'comm_use_subset/comm_use_subset/',
        'noncomm_use_subset/noncomm_use_subset/'
        ]

result_path = '/home/santosh/Work/Datasets/CORD-19-sentences/'



In [2]:
import glob
import json
from tqdm import tqdm
import pandas as pd

In [3]:
# get all the jsosn

all_jsons =[]
for json_path in paths:
     all_jsons.extend(sorted(glob.glob(root_path+json_path + '*.json*')))

# all_json = glob.glob(f'{root_path}/**/*.json', recursive=True)

In [4]:
all_jsons[0]

'/home/santosh/Work/Datasets/CORD-19-research-challenge/biorxiv_medrxiv/biorxiv_medrxiv/0015023cc06b5362d332b3baf348d11567ca2fbb.json'

In [5]:
meta_df = pd.read_csv('/home/santosh/Work/Datasets/CORD-19-research-challenge/metadata.csv')

In [6]:
meta_df.columns

Index(['sha', 'source_x', 'title', 'doi', 'pmcid', 'pubmed_id', 'license',
       'abstract', 'publish_time', 'authors', 'journal',
       'Microsoft Academic Paper ID', 'WHO #Covidence', 'has_full_text',
       'full_text_file'],
      dtype='object')

In [7]:
selected_columns = ['sha', 'doi', 'pmcid', 'pubmed_id', 'publish_time'] # sha is paper id here

In [8]:
meta_ref_df = meta_df[selected_columns]
meta_ref_df = meta_ref_df[meta_ref_df['sha'].notna()]


In [10]:
# extract sentences with doi


for each_json_file in tqdm(all_jsons):
    with open(each_json_file) as json_file:
        data = json.load(json_file)
        
        try:
            source = meta_ref_df[meta_ref_df['sha'].str.contains(data['paper_id'])]['doi'].values.tolist()[0]
        except:
            source =''
            
        full_text = []

        try:
            for each_text in data['metadata']:
                full_text.append(each_text['title']+' ---'+source+'---')
        except:
                pass 

        
        try:
            for each_text in data['abstract']:
                full_text.append(each_text['text']+' ---'+source+'---')
        except:
                pass

        try:
            for each_text in data['body_text']:
                full_text.append(each_text['text']+' ---'+source+'---')
        except:
                pass
            
        try:
            for each_text in data['ref_entries']:
                full_text.append(data['ref_entries'][each_text]['text']+' ---'+source+'---')
        except:
                pass    
        
        with open(result_path+each_json_file.split('/')[-1][:-5]+'.txt', 'a') as writer:
            for each_para in full_text:
                    if 'doi:' not in each_para and 'word count' not in each_para and 'All rights reserved' not in each_para and 'No reuse allowed without permission' not in each_para:
                        writer.write(each_para+'\n')     

100%|██████████| 12356/12356 [03:33<00:00, 57.96it/s]


# Generate Fasttext

In [1]:
# Here I am using gensim for learning the Fast Text word embeddings from the CORD-19
import nltk
import logging
import os
# download NLTK basic stopwords
import nltk
from nltk.corpus import stopwords

from nltk.tokenize.treebank import TreebankWordDetokenizer, TreebankWordTokenizer


# start the log
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',level=logging.INFO)


def clean_my_text(full_text_):
    stopset = set(stopwords.words('english')) #| set(string.punctuation)
    tokens = TreebankWordTokenizer().tokenize(full_text_.split('---10')[0])
    cleanup = [token for token in tokens if token not in stopset and len(token) > 1]
    return cleanup


# get the current working directory and file
data_dir_path = '/home/santosh/Work/Datasets/CORD-19-text/'
result_dir_path = '/home/santosh/Work/models/word2vec/CORD-19/'

In [2]:
class SentenceClass(object):
    def __init__(self, dirname):
        self.dirname = dirname

    def __iter__(self):
        for fname in os.listdir(self.dirname):
#             print(fname)
            for line in open(os.path.join(self.dirname, fname), 'r'):
                yield clean_my_text(line)


# iterate one line at a time and learn embeddings
paragraphs = SentenceClass(data_dir_path)  # a memory-friendly iterator

In [3]:
from gensim.models.fasttext import FastText

model = FastText(size=200,
    window=5,
    min_count=5,
    workers=4, iter=10)

model.build_vocab(sentences=paragraphs)
training_examples_count = model.corpus_count







2020-03-31 15:53:44,068 : INFO : resetting layer weights
2020-03-31 15:53:53,916 : INFO : collecting all words and their counts
2020-03-31 15:53:54,103 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-03-31 15:53:57,895 : INFO : PROGRESS: at sentence #10000, processed 161591 words, keeping 20240 word types
2020-03-31 15:54:00,981 : INFO : PROGRESS: at sentence #20000, processed 311196 words, keeping 32154 word types
2020-03-31 15:54:04,477 : INFO : PROGRESS: at sentence #30000, processed 465308 words, keeping 42013 word types
2020-03-31 15:54:07,642 : INFO : PROGRESS: at sentence #40000, processed 624049 words, keeping 50336 word types
2020-03-31 15:54:11,056 : INFO : PROGRESS: at sentence #50000, processed 779830 words, keeping 57607 word types
2020-03-31 15:54:14,231 : INFO : PROGRESS: at sentence #60000, processed 939397 words, keeping 64221 word types
2020-03-31 15:54:17,536 : INFO : PROGRESS: at sentence #70000, processed 1097969 words, keeping 71329

2020-03-31 15:57:47,778 : INFO : PROGRESS: at sentence #710000, processed 11079456 words, keeping 343397 word types
2020-03-31 15:57:51,041 : INFO : PROGRESS: at sentence #720000, processed 11228001 words, keeping 346328 word types
2020-03-31 15:57:54,509 : INFO : PROGRESS: at sentence #730000, processed 11388323 words, keeping 349740 word types
2020-03-31 15:57:58,098 : INFO : PROGRESS: at sentence #740000, processed 11545091 words, keeping 352655 word types
2020-03-31 15:58:01,294 : INFO : PROGRESS: at sentence #750000, processed 11697620 words, keeping 356085 word types
2020-03-31 15:58:04,788 : INFO : PROGRESS: at sentence #760000, processed 11857529 words, keeping 359580 word types
2020-03-31 15:58:07,991 : INFO : PROGRESS: at sentence #770000, processed 12013964 words, keeping 362869 word types
2020-03-31 15:58:11,192 : INFO : PROGRESS: at sentence #780000, processed 12167589 words, keeping 366068 word types
2020-03-31 15:58:14,561 : INFO : PROGRESS: at sentence #790000, processe

2020-03-31 16:01:46,436 : INFO : PROGRESS: at sentence #1420000, processed 22264160 words, keeping 574092 word types
2020-03-31 16:01:49,715 : INFO : PROGRESS: at sentence #1430000, processed 22417417 words, keeping 576704 word types
2020-03-31 16:01:52,703 : INFO : PROGRESS: at sentence #1440000, processed 22566289 words, keeping 579322 word types
2020-03-31 16:01:56,558 : INFO : PROGRESS: at sentence #1450000, processed 22724147 words, keeping 582070 word types
2020-03-31 16:01:59,855 : INFO : PROGRESS: at sentence #1460000, processed 22877127 words, keeping 584790 word types
2020-03-31 16:02:03,003 : INFO : PROGRESS: at sentence #1470000, processed 23033897 words, keeping 587364 word types
2020-03-31 16:02:06,336 : INFO : PROGRESS: at sentence #1480000, processed 23195707 words, keeping 590011 word types
2020-03-31 16:02:09,784 : INFO : PROGRESS: at sentence #1490000, processed 23353942 words, keeping 592683 word types
2020-03-31 16:02:12,948 : INFO : PROGRESS: at sentence #1500000,

2020-03-31 16:05:39,834 : INFO : PROGRESS: at sentence #2130000, processed 33360634 words, keeping 748560 word types
2020-03-31 16:05:43,105 : INFO : PROGRESS: at sentence #2140000, processed 33520584 words, keeping 750843 word types
2020-03-31 16:05:46,552 : INFO : PROGRESS: at sentence #2150000, processed 33681648 words, keeping 753153 word types
2020-03-31 16:05:49,983 : INFO : PROGRESS: at sentence #2160000, processed 33843099 words, keeping 755577 word types
2020-03-31 16:05:53,124 : INFO : PROGRESS: at sentence #2170000, processed 33991689 words, keeping 758156 word types
2020-03-31 16:05:56,305 : INFO : PROGRESS: at sentence #2180000, processed 34146978 words, keeping 760799 word types
2020-03-31 16:05:59,800 : INFO : PROGRESS: at sentence #2190000, processed 34313596 words, keeping 763429 word types
2020-03-31 16:06:03,025 : INFO : PROGRESS: at sentence #2200000, processed 34469713 words, keeping 765652 word types
2020-03-31 16:06:06,294 : INFO : PROGRESS: at sentence #2210000,

In [4]:
model.train(paragraphs, total_examples=training_examples_count, epochs=model.iter)
logging.info("Done training data files")

/home/santosh/anaconda3/envs/scispacy/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """Entry point for launching an IPython kernel.
2020-03-31 16:07:16,979 : INFO : training model with 4 workers on 165124 vocabulary and 200 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2020-03-31 16:07:18,157 : INFO : EPOCH 1 - PROGRESS: at 0.09% examples, 24687 words/s, in_qsize 0, out_qsize 0
2020-03-31 16:07:19,372 : INFO : EPOCH 1 - PROGRESS: at 0.24% examples, 36275 words/s, in_qsize 0, out_qsize 0
2020-03-31 16:07:20,468 : INFO : EPOCH 1 - PROGRESS: at 0.37% examples, 38708 words/s, in_qsize 0, out_qsize 0
2020-03-31 16:07:21,490 : INFO : EPOCH 1 - PROGRESS: at 0.51% examples, 40599 words/s, in_qsize 0, out_qsize 0
2020-03-31 16:07:22,533 : INFO : EPOCH 1 - PROGRESS: at 0.66% examples, 41582 words/s, in_qsize 0, out_qsize 0
2020-03-31 16:07:23,681 : INFO : EPOCH 1 - P

2020-03-31 16:08:35,244 : INFO : EPOCH 1 - PROGRESS: at 9.98% examples, 44396 words/s, in_qsize 0, out_qsize 0
2020-03-31 16:08:36,311 : INFO : EPOCH 1 - PROGRESS: at 10.11% examples, 44406 words/s, in_qsize 0, out_qsize 0
2020-03-31 16:08:37,493 : INFO : EPOCH 1 - PROGRESS: at 10.26% examples, 44345 words/s, in_qsize 0, out_qsize 0
2020-03-31 16:08:38,507 : INFO : EPOCH 1 - PROGRESS: at 10.42% examples, 44380 words/s, in_qsize 0, out_qsize 0
2020-03-31 16:08:39,670 : INFO : EPOCH 1 - PROGRESS: at 10.56% examples, 44335 words/s, in_qsize 0, out_qsize 0
2020-03-31 16:08:40,824 : INFO : EPOCH 1 - PROGRESS: at 10.70% examples, 44297 words/s, in_qsize 0, out_qsize 0
2020-03-31 16:08:41,870 : INFO : EPOCH 1 - PROGRESS: at 10.83% examples, 44319 words/s, in_qsize 0, out_qsize 0
2020-03-31 16:08:42,935 : INFO : EPOCH 1 - PROGRESS: at 10.97% examples, 44327 words/s, in_qsize 0, out_qsize 0
2020-03-31 16:08:43,985 : INFO : EPOCH 1 - PROGRESS: at 11.11% examples, 44343 words/s, in_qsize 0, out_q

2020-03-31 16:09:56,004 : INFO : EPOCH 1 - PROGRESS: at 20.27% examples, 44100 words/s, in_qsize 0, out_qsize 0
2020-03-31 16:09:57,052 : INFO : EPOCH 1 - PROGRESS: at 20.40% examples, 44111 words/s, in_qsize 0, out_qsize 0
2020-03-31 16:09:58,187 : INFO : EPOCH 1 - PROGRESS: at 20.54% examples, 44099 words/s, in_qsize 0, out_qsize 0
2020-03-31 16:09:59,232 : INFO : EPOCH 1 - PROGRESS: at 20.68% examples, 44111 words/s, in_qsize 0, out_qsize 0
2020-03-31 16:10:00,277 : INFO : EPOCH 1 - PROGRESS: at 20.81% examples, 44122 words/s, in_qsize 0, out_qsize 0
2020-03-31 16:10:01,315 : INFO : EPOCH 1 - PROGRESS: at 20.95% examples, 44134 words/s, in_qsize 0, out_qsize 0
2020-03-31 16:10:02,398 : INFO : EPOCH 1 - PROGRESS: at 21.08% examples, 44131 words/s, in_qsize 0, out_qsize 0
2020-03-31 16:10:03,503 : INFO : EPOCH 1 - PROGRESS: at 21.22% examples, 44127 words/s, in_qsize 0, out_qsize 0
2020-03-31 16:10:04,552 : INFO : EPOCH 1 - PROGRESS: at 21.35% examples, 44136 words/s, in_qsize 0, out_

2020-03-31 16:11:16,813 : INFO : EPOCH 1 - PROGRESS: at 30.45% examples, 43881 words/s, in_qsize 0, out_qsize 0
2020-03-31 16:11:17,936 : INFO : EPOCH 1 - PROGRESS: at 30.59% examples, 43876 words/s, in_qsize 0, out_qsize 0
2020-03-31 16:11:19,064 : INFO : EPOCH 1 - PROGRESS: at 30.73% examples, 43869 words/s, in_qsize 0, out_qsize 0
2020-03-31 16:11:20,134 : INFO : EPOCH 1 - PROGRESS: at 30.87% examples, 43873 words/s, in_qsize 0, out_qsize 0
2020-03-31 16:11:21,185 : INFO : EPOCH 1 - PROGRESS: at 31.01% examples, 43882 words/s, in_qsize 0, out_qsize 0
2020-03-31 16:11:22,330 : INFO : EPOCH 1 - PROGRESS: at 31.16% examples, 43873 words/s, in_qsize 0, out_qsize 0
2020-03-31 16:11:23,440 : INFO : EPOCH 1 - PROGRESS: at 31.30% examples, 43870 words/s, in_qsize 0, out_qsize 0
2020-03-31 16:11:24,622 : INFO : EPOCH 1 - PROGRESS: at 31.46% examples, 43893 words/s, in_qsize 0, out_qsize 0
2020-03-31 16:11:25,674 : INFO : EPOCH 1 - PROGRESS: at 31.59% examples, 43900 words/s, in_qsize 0, out_

2020-03-31 16:12:39,184 : INFO : EPOCH 1 - PROGRESS: at 40.49% examples, 43498 words/s, in_qsize 0, out_qsize 0
2020-03-31 16:12:40,222 : INFO : EPOCH 1 - PROGRESS: at 40.55% examples, 43417 words/s, in_qsize 0, out_qsize 0
2020-03-31 16:12:41,259 : INFO : EPOCH 1 - PROGRESS: at 40.63% examples, 43368 words/s, in_qsize 0, out_qsize 0
2020-03-31 16:12:42,451 : INFO : EPOCH 1 - PROGRESS: at 40.77% examples, 43356 words/s, in_qsize 0, out_qsize 0
2020-03-31 16:12:43,606 : INFO : EPOCH 1 - PROGRESS: at 40.91% examples, 43349 words/s, in_qsize 0, out_qsize 0
2020-03-31 16:12:44,785 : INFO : EPOCH 1 - PROGRESS: at 41.05% examples, 43339 words/s, in_qsize 0, out_qsize 0
2020-03-31 16:12:45,826 : INFO : EPOCH 1 - PROGRESS: at 41.19% examples, 43344 words/s, in_qsize 0, out_qsize 0
2020-03-31 16:12:46,981 : INFO : EPOCH 1 - PROGRESS: at 41.33% examples, 43338 words/s, in_qsize 0, out_qsize 0
2020-03-31 16:12:48,069 : INFO : EPOCH 1 - PROGRESS: at 41.47% examples, 43339 words/s, in_qsize 0, out_

...

2020-03-31 17:57:58,687 : INFO : EPOCH 10 - PROGRESS: at 47.47% examples, 48243 words/s, in_qsize 0, out_qsize 0
2020-03-31 17:57:59,709 : INFO : EPOCH 10 - PROGRESS: at 47.61% examples, 48239 words/s, in_qsize 0, out_qsize 0
2020-03-31 17:58:00,834 : INFO : EPOCH 10 - PROGRESS: at 47.75% examples, 48220 words/s, in_qsize 0, out_qsize 0
2020-03-31 17:58:01,840 : INFO : EPOCH 10 - PROGRESS: at 47.90% examples, 48219 words/s, in_qsize 0, out_qsize 0
2020-03-31 17:58:03,008 : INFO : EPOCH 10 - PROGRESS: at 48.06% examples, 48222 words/s, in_qsize 0, out_qsize 0
2020-03-31 17:58:04,061 : INFO : EPOCH 10 - PROGRESS: at 48.19% examples, 48280 words/s, in_qsize 0, out_qsize 0
2020-03-31 17:58:05,064 : INFO : EPOCH 10 - PROGRESS: at 48.33% examples, 48320 words/s, in_qsize 0, out_qsize 0
2020-03-31 17:58:06,132 : INFO : EPOCH 10 - PROGRESS: at 48.48% examples, 48353 words/s, in_qsize 0, out_qsize 0
2020-03-31 17:58:07,294 : INFO : EPOCH 10 - PROGRESS: at 48.64% examples, 48357 words/s, in_qsiz

2020-03-31 17:59:19,524 : INFO : EPOCH 10 - PROGRESS: at 58.74% examples, 48371 words/s, in_qsize 0, out_qsize 0
2020-03-31 17:59:20,667 : INFO : EPOCH 10 - PROGRESS: at 58.90% examples, 48377 words/s, in_qsize 0, out_qsize 0
2020-03-31 17:59:21,791 : INFO : EPOCH 10 - PROGRESS: at 59.06% examples, 48383 words/s, in_qsize 0, out_qsize 0
2020-03-31 17:59:22,968 : INFO : EPOCH 10 - PROGRESS: at 59.23% examples, 48384 words/s, in_qsize 0, out_qsize 0
2020-03-31 17:59:23,975 : INFO : EPOCH 10 - PROGRESS: at 59.37% examples, 48382 words/s, in_qsize 0, out_qsize 0
2020-03-31 17:59:24,977 : INFO : EPOCH 10 - PROGRESS: at 59.50% examples, 48381 words/s, in_qsize 0, out_qsize 0
2020-03-31 17:59:25,981 : INFO : EPOCH 10 - PROGRESS: at 59.65% examples, 48381 words/s, in_qsize 0, out_qsize 0
2020-03-31 17:59:27,014 : INFO : EPOCH 10 - PROGRESS: at 59.81% examples, 48376 words/s, in_qsize 0, out_qsize 0
2020-03-31 17:59:28,019 : INFO : EPOCH 10 - PROGRESS: at 59.95% examples, 48375 words/s, in_qsiz

2020-03-31 18:00:40,469 : INFO : EPOCH 10 - PROGRESS: at 70.17% examples, 48387 words/s, in_qsize 0, out_qsize 0
2020-03-31 18:00:41,628 : INFO : EPOCH 10 - PROGRESS: at 70.33% examples, 48390 words/s, in_qsize 0, out_qsize 0
2020-03-31 18:00:42,770 : INFO : EPOCH 10 - PROGRESS: at 70.49% examples, 48396 words/s, in_qsize 0, out_qsize 0
2020-03-31 18:00:43,925 : INFO : EPOCH 10 - PROGRESS: at 70.66% examples, 48399 words/s, in_qsize 0, out_qsize 0
2020-03-31 18:00:44,934 : INFO : EPOCH 10 - PROGRESS: at 70.80% examples, 48398 words/s, in_qsize 0, out_qsize 0
2020-03-31 18:00:46,079 : INFO : EPOCH 10 - PROGRESS: at 70.96% examples, 48401 words/s, in_qsize 0, out_qsize 0
2020-03-31 18:00:47,288 : INFO : EPOCH 10 - PROGRESS: at 71.14% examples, 48398 words/s, in_qsize 0, out_qsize 0
2020-03-31 18:00:48,360 : INFO : EPOCH 10 - PROGRESS: at 71.29% examples, 48390 words/s, in_qsize 0, out_qsize 0
2020-03-31 18:00:49,530 : INFO : EPOCH 10 - PROGRESS: at 71.45% examples, 48392 words/s, in_qsiz

2020-03-31 18:02:03,260 : INFO : EPOCH 10 - PROGRESS: at 81.74% examples, 48409 words/s, in_qsize 0, out_qsize 0
2020-03-31 18:02:04,285 : INFO : EPOCH 10 - PROGRESS: at 81.88% examples, 48406 words/s, in_qsize 0, out_qsize 0
2020-03-31 18:02:05,478 : INFO : EPOCH 10 - PROGRESS: at 82.05% examples, 48407 words/s, in_qsize 0, out_qsize 0
2020-03-31 18:02:06,662 : INFO : EPOCH 10 - PROGRESS: at 82.21% examples, 48408 words/s, in_qsize 0, out_qsize 0
2020-03-31 18:02:07,702 : INFO : EPOCH 10 - PROGRESS: at 82.36% examples, 48404 words/s, in_qsize 0, out_qsize 0
2020-03-31 18:02:08,713 : INFO : EPOCH 10 - PROGRESS: at 82.49% examples, 48402 words/s, in_qsize 0, out_qsize 0
2020-03-31 18:02:09,869 : INFO : EPOCH 10 - PROGRESS: at 82.65% examples, 48405 words/s, in_qsize 0, out_qsize 0
2020-03-31 18:02:10,876 : INFO : EPOCH 10 - PROGRESS: at 82.80% examples, 48404 words/s, in_qsize 0, out_qsize 0
2020-03-31 18:02:11,903 : INFO : EPOCH 10 - PROGRESS: at 82.94% examples, 48401 words/s, in_qsiz

2020-03-31 18:03:24,092 : INFO : EPOCH 10 - PROGRESS: at 92.85% examples, 48326 words/s, in_qsize 0, out_qsize 0
2020-03-31 18:03:25,254 : INFO : EPOCH 10 - PROGRESS: at 93.01% examples, 48328 words/s, in_qsize 0, out_qsize 0
2020-03-31 18:03:26,260 : INFO : EPOCH 10 - PROGRESS: at 93.15% examples, 48328 words/s, in_qsize 0, out_qsize 0
2020-03-31 18:03:27,413 : INFO : EPOCH 10 - PROGRESS: at 93.30% examples, 48330 words/s, in_qsize 0, out_qsize 0
2020-03-31 18:03:28,423 : INFO : EPOCH 10 - PROGRESS: at 93.44% examples, 48329 words/s, in_qsize 0, out_qsize 0
2020-03-31 18:03:29,582 : INFO : EPOCH 10 - PROGRESS: at 93.60% examples, 48332 words/s, in_qsize 0, out_qsize 0
2020-03-31 18:03:30,623 : INFO : EPOCH 10 - PROGRESS: at 93.76% examples, 48328 words/s, in_qsize 0, out_qsize 0
2020-03-31 18:03:31,690 : INFO : EPOCH 10 - PROGRESS: at 93.90% examples, 48322 words/s, in_qsize 0, out_qsize 0
2020-03-31 18:03:32,886 : INFO : EPOCH 10 - PROGRESS: at 94.07% examples, 48322 words/s, in_qsiz

In [6]:
model.save(result_dir_path+'CORD-19-FT_200d_5w_10i_5mc_sentence.model')


2020-03-31 18:11:30,190 : INFO : saving FastText object under /home/santosh/Work/models/word2vec/CORD-19/CORD-19-FT_200d_5w_10i_5mc_sentence.model, separately None
2020-03-31 18:11:30,192 : INFO : storing np array 'vectors' to /home/santosh/Work/models/word2vec/CORD-19/CORD-19-FT_200d_5w_10i_5mc_sentence.model.wv.vectors.npy
2020-03-31 18:11:30,491 : INFO : not storing attribute vectors_norm
2020-03-31 18:11:30,492 : INFO : storing np array 'vectors_vocab' to /home/santosh/Work/models/word2vec/CORD-19/CORD-19-FT_200d_5w_10i_5mc_sentence.model.wv.vectors_vocab.npy
2020-03-31 18:11:30,554 : INFO : storing np array 'vectors_ngrams' to /home/santosh/Work/models/word2vec/CORD-19/CORD-19-FT_200d_5w_10i_5mc_sentence.model.wv.vectors_ngrams.npy
2020-03-31 18:11:54,088 : INFO : not storing attribute vectors_ngrams_norm
2020-03-31 18:11:54,090 : INFO : not storing attribute vectors_vocab_norm
2020-03-31 18:11:54,091 : INFO : not storing attribute buckets_word
2020-03-31 18:11:54,093 : INFO : sto

In [5]:
model.wv.most_similar('SARS-CoV-2', topn=10)

2020-03-31 18:11:18,124 : INFO : precomputing L2-norms of word weight vectors
2020-03-31 18:11:18,331 : INFO : precomputing L2-norms of ngram weight vectors


[('2019-nCoV/SARS-CoV-2', 0.9531718492507935),
 ('SARS-CoV-', 0.9292833805084229),
 ('SARS-CoV-cpsR-19', 0.9127612113952637),
 ('SARS-CoV-ΔE', 0.9096049070358276),
 ('SAR-CoV-2', 0.9028514623641968),
 ('SARS-CoV-1', 0.9025784730911255),
 ('SARS-CoV-or', 0.9000788927078247),
 ('2019-nCoV', 0.8876548409461975),
 ('SARS-CoV-ΔE-8a-dup', 0.8799664974212646),
 ('SARS-nCoV', 0.8798669576644897)]

# Generate Paragraph Embeddings

In [18]:
# extract sentences with doi and year after 2017. We are only interested in COVID-19 papers

result_path = '/home/santosh/Work/Datasets/COVID-CORD-19_paragraphs/'
for each_json_file in tqdm(all_jsons):
    with open(each_json_file) as json_file:
        data = json.load(json_file)
        
        try:
            date_t = meta_ref_df[meta_ref_df['sha'].str.contains(data['paper_id'])]['publish_time'].values.tolist()[0]
            
            if parser.parse(date_t).year>2017:
                try:
                    source = meta_ref_df[meta_ref_df['sha'].str.contains(data['paper_id'])]['doi'].values.tolist()[0]
                except:
                    source =''

                full_text = []

                try:
                    for each_text in data['metadata']:
                        full_text.append(each_text['title']+' ---'+source+'---')
                except:
                        pass 


                try:
                    for each_text in data['abstract']:
                        full_text.append(each_text['text']+' ---'+source+'---')
                except:
                        pass

                try:
                    for each_text in data['body_text']:
                        full_text.append(each_text['text']+' ---'+source+'---')
                except:
                        pass

                try:
                    for each_text in data['ref_entries']:
                        full_text.append(data['ref_entries'][each_text]['text']+' ---'+source+'---')
                except:
                        pass    

                with open(result_path+each_json_file.split('/')[-1][:-5]+'.txt', 'a') as writer:
                    for each_para in full_text:
                            if 'doi:' not in each_para and 'word count' not in each_para and 'All rights reserved' not in each_para and 'No reuse allowed without permission' not in each_para:
                                writer.write(each_para+'\n')     
        except:
            pass

100%|██████████| 12356/12356 [04:37<00:00, 44.46it/s]
